Import libraries

In [ ]:
import numpy as np

Create a simple dataset of text sequences

In [ ]:
# @title Default title text
np.random.seed(42)

def generate_dataset(num_words=100, max_word_size=10):

    sequences = []

    for _ in range (num_words):

        random_number =  np.random.randint(1, max_word_size)
        sample = ['s'] * random_number + ['t'] * random_number +  ['EOS']
        sequences.append(sample)

    return sequences

num_words=100
max_word_size=10

vocabulary = generate_dataset(num_words, max_word_size)
print(vocabulary[88])

[['s', 's', 's', 's', 's', 's', 's', 't', 't', 't', 't', 't', 't', 't', 'EOS'], ['s', 's', 's', 's', 't', 't', 't', 't', 'EOS'], ['s', 's', 's', 's', 's', 's', 's', 's', 't', 't', 't', 't', 't', 't', 't', 't', 'EOS'], ['s', 's', 's', 's', 's', 't', 't', 't', 't', 't', 'EOS'], ['s', 's', 's', 's', 's', 's', 's', 't', 't', 't', 't', 't', 't', 't', 'EOS'], ['s', 's', 's', 't', 't', 't', 'EOS'], ['s', 's', 's', 's', 's', 's', 's', 't', 't', 't', 't', 't', 't', 't', 'EOS'], ['s', 's', 's', 's', 's', 's', 's', 's', 't', 't', 't', 't', 't', 't', 't', 't', 'EOS'], ['s', 's', 's', 's', 's', 't', 't', 't', 't', 't', 'EOS'], ['s', 's', 's', 's', 't', 't', 't', 't', 'EOS'], ['s', 's', 's', 's', 's', 's', 's', 's', 't', 't', 't', 't', 't', 't', 't', 't', 'EOS'], ['s', 's', 's', 's', 's', 's', 's', 's', 't', 't', 't', 't', 't', 't', 't', 't', 'EOS'], ['s', 's', 's', 't', 't', 't', 'EOS'], ['s', 's', 's', 's', 's', 's', 't', 't', 't', 't', 't', 't', 'EOS'], ['s', 's', 's', 's', 's', 't', 't', 't', 't

Perform one-hot encoding on the generated vocabulary

In [ ]:
def seq_to_dict(vocabulary):

    word_to_index = {}
    index_to_word = {}

    str_vocabulary = [''.join(word) for word in vocabulary]

    # remove duplicate words in the dictionary
    unique_vocabulary = set(str_vocabulary)
    unique_vocabulary = list(unique_vocabulary)

    # find the word with the maximum length in the vocabulary
    max_word_size = max(unique_vocabulary, key=len)
    # compute the maximum word length
    max_word_size = int((len(max_word_size) - 3 ) / 2)

    # create a diagonal list of lists
    encoding = [['0' if i != j else '1' for j in range(max_word_size)] for i in range(max_word_size)]
    encoding = [''.join(row) for row in encoding]

    for word in unique_vocabulary:

        word_length = int((len(word) - 3 ) / 2)
        word_to_index[word] = encoding[word_length-1]
        index_to_word[encoding[word_length-1]] = word

    return word_to_index, index_to_word

word_to_index, index_to_word = seq_to_dict(vocabulary)

print(word_to_index)
print(index_to_word)

index_to_word = 1